<a href="https://colab.research.google.com/github/zahraDehghanian97/Poetry_Generator/blob/master/Char_Poem_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pickle
from nltk.metrics import accuracy ,ConfusionMatrix 
from nltk.translate.bleu_score import sentence_bleu

In [2]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
embedding_dim = 256
rnn_units = 1024
seq_length = 1000

# make data ready

In [3]:
filepath = "/content/drive/MyDrive/Colab Notebooks/my_shahname_represntation.txt"
with open(filepath, "rb") as f:
    corpus , test = pickle.load(f)
corpus = corpus.replace("\t"," \t ").replace("\n", " \n ")

In [4]:
vocab = sorted(set(corpus))
vocab_size = len(vocab)
print(len(vocab), "chars")
char2idx = {u:i for i, u in enumerate(vocab)}
print(char2idx)
idx2char = np.array(vocab)

48 chars
{'\t': 0, '\n': 1, ' ': 2, '(': 3, ')': 4, '«': 5, '»': 6, '،': 7, '؟': 8, 'ء': 9, 'آ': 10, 'أ': 11, 'ؤ': 12, 'ئ': 13, 'ا': 14, 'ب': 15, 'ت': 16, 'ث': 17, 'ج': 18, 'ح': 19, 'خ': 20, 'د': 21, 'ذ': 22, 'ر': 23, 'ز': 24, 'س': 25, 'ش': 26, 'ص': 27, 'ض': 28, 'ط': 29, 'ظ': 30, 'ع': 31, 'غ': 32, 'ف': 33, 'ق': 34, 'ل': 35, 'م': 36, 'ن': 37, 'ه': 38, 'و': 39, 'ٔ': 40, 'پ': 41, 'چ': 42, 'ژ': 43, 'ک': 44, 'گ': 45, 'ی': 46, '\u200c': 47}


In [5]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

text_as_int = np.array([char2idx[c] for c in corpus])
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# LSTM Model

In [6]:
def create_model_lstm(vocab_size=vocab_size, embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE):
  model = keras.Sequential()
  model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,batch_input_shape=[batch_size, None]))
  model.add(tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform',recurrent_regularizer=keras.regularizers.l2(0.01)))
  model.add(tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform',recurrent_regularizer=keras.regularizers.l2(0.01)))
  model.add(tf.keras.layers.Dense(vocab_size))
  return model

In [7]:
# lstm_model = create_model_lstm()
# lstm_model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
# history = lstm_model.fit(dataset, epochs=50)
main_lstm_model = create_model_lstm(batch_size=1)
# main_lstm_model.set_weights(lstm_model.get_weights())
main_lstm_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/char_lstm.h5')
main_lstm_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            12288     
_________________________________________________________________
lstm_2 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
lstm_3 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
dense_1 (Dense)              (1, None, 48)             49200     
Total params: 13,701,168
Trainable params: 13,701,168
Non-trainable params: 0
_________________________________________________________________


In [8]:
def generate_text(model, start_string):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])
  return (start_string + ''.join(text_generated))

In [9]:
print(generate_text(main_lstm_model, start_string="چنین گفت رستم به اسفندیار"))
main_lstm_model.save("/content/drive/MyDrive/Colab Notebooks/char_lstm.h5")

چنین گفت رستم به اسفندیار 	 به پیش اندرون پیل و کلاه مرد 
 درختیست چینی و کوتاه مرا 	 نشاید به رودی ارش روزگار 
 سگیندر نهادی نداریم شکن 	 سپاهی تویم روشنوای من 
 همی برد سوی شهرو آنچهر 	 سپه بیرون آید ز دشمن و رود 
 نبرد سوی تیر دیو را نبرد 	 بزرگی و چوخسرو شوید خویش 
 وزین پس نشاند کز پیش خرد 	 ممانم به دل بود نیرو بود 
 یکی را زبانها ز بردند هر 	 برفت از کاری پرخرگاه سر 
 چو جهان سرشان را هست باد 	 که ای نامور پر ما را ستم پند 
 پیام فرزند بر تا بباغ 	 دل مرد را بر بزم از خوشاب 
 نشست از پلنگان او را بدیم 	 ببینی ز گوهر نگذاشتیم 
 وگر این زمان هر دام سوی مرد 	 که این در زداران تو جستن نیست 
 ز پیوند یکی تاج با چیز و فر 	 چو باشی مرا نایزدش دست 
 ولیکن شهره ترساندی بدید 	 به فتراک بر شطرنج آورد 
 ز رها دل این سخن راندمست 	 بدین کار پیش گیو منشو شود 
 بریشان او دینار و جوش وبوم 	 بگرم اندر آرند نیز پدرود 
 ز مرغان سوی را خرام ناسزا 	 که دارد دل نبردارد گر راست 
 تو اندر زمان لشکر بیرونیست 	 جز وخواند آید یکسر به سرم 
 چو بی‌دود بر پنج خویش را 	 سخن رفت هیون زودست بسوز 
 گزین کز هر پای

# Test

In [22]:
BLEU_scores = []
accuracy_scores = []
poem = test[0]
start = poem[:25]
generated_poem = generate_text(main_lstm_model, start_string=start)
BLEU_scores.append(sentence_bleu(poem, generated_poem))
len_min = min(len(poem),len(generated_poem))
accuracy_scores.append(accuracy(poem[:len_min], generated_poem[:len_min]))
print("-----------------------")
print("start sentence : ",start)
print(generated_poem)
print("BLEU score = ",BLEU_scores[-1])
print("Accuracy score = ",accuracy_scores[-1])
print("Confusion matrix =")
print(ConfusionMatrix(poem[:len_min], generated_poem[:len_min]))

counter = 0
for poem in test :
  counter+=1
  start = poem[:25]
  generated_poem = generate_text(main_lstm_model, start_string=start)
  BLEU_scores.append(sentence_bleu(poem, generated_poem))
  len_min = min(len(poem),len(generated_poem))
  accuracy_scores.append(accuracy(poem[:len_min], generated_poem[:len_min]))
  print("-----------------------")
  print("sentence number : ",counter)
  print("BLEU score = ",BLEU_scores[-1])
  print("Accuracy score = ",accuracy_scores[-1])

print("<<------------final report----------->>")
print("number of test set = ",len(test))
print("mean BLEU score = ",np.mean(BLEU_scores))
print("mean Accuracy score = ",np.mean(accuracy_scores))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


-----------------------
start sentence :  چو لشکر شد از خواسته بی‌ن
چو لشکر شد از خواسته بی‌نما 
 بپذرفت و پرمور جایگاه 
 تو با آب و ایرانیان سخن 	 همه روی شادمان و نهادن 
 به هنرسوی یال افراسیاب 	 که چهارم از پیش آواز نیز 
 مرا گفت کان جنگ رستمست 
 چو در جنگ پادشاهی خاست 	 از تخم گیرد کشیدن سخن 
 تو بر نه سیری نیار مردان 	 که نام گرایی و فرهنگ و ناز 
 نشست از بر خواسته فرمان 	 که بیند کز کاری ویلان مها 
 گر از تو روان آژی تیره‌روان 	 ترا با فرهانیده راز خوان 
 کلاه وسی پسند برارگاه 	 بغرفت ما رامش گرز هروان 
 نهادند زو کین سالار یو 	 ز لشکر ز کردار قیصر برافروختند 
 وزان پس برز و عنبر زین 	 ز رامشگران چو درفش سیاه 
 فروماند بر شهریار جهان 	 بزرگی درنده زان کار خویش 
 به شاهی که از تخت لعلت مهر 	 و گرزن تر منستا خورشید 
 اگر تابوتانگان خود نگاه 	 بخم کمرهٔ خویش را بکشت 
 چو گشتی برآمد دوان روی کار 	 بسی روز خاک ترا زین برکند 
 چه گویم همی دارم بدان زدان 	 نماند به گیتی ز کار رساد 
 به دلت گردش از رسم آزمک 	 یکی چرب تیر فرمودش نژاد 
 بزرگان و جانم بتاجاست کوت 	 به میدان گر با اختر خوارخ